# IMPORT WHOLE REQUIREMENTS

In [1]:
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import sys
 
from datetime import datetime
from numpy import genfromtxt
from tensorflow import keras

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model
# https://www.tensorflow.org/tutorials/keras/save_and_load
# https://www.tensorflow.org/guide/keras/save_and_serialize

# TIMER

In [3]:
import time

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Executed time: {elapsed_time:0.4f} seconds")

# Load Data I/O

In [4]:
A = np.array([])
B = np.array([])

# df = genfromtxt('./drive/MyDrive/IPYNB/Datasets/dataset_trade.csv', delimiter=',')
df = genfromtxt('./dataset_trade_2k.csv', delimiter=',')
print(len(df))
for i in range(len(df)):
  if(i>1):
    A = np.append(A, [ df[i][13], df[i][14], df[i][15], df[i][16], df[i][17] ])
    B = np.append(B, [ df[i][23], df[i][24], df[i][25], df[i][26], df[i][27] ])

    x_train = np.reshape(A, (-1, 5, ))
    y_train = np.reshape(B, (-1, 5, ))


2002


In [5]:
print(len(x_train))
print(x_train.shape)
print(x_train[0])

print(len(y_train))
print(y_train.shape)
print(y_train[0])

2000
(2000, 5)
[0.9995052  0.83333333 0.41935484 0.25       0.68333333]
2000
(2000, 5)
[0.96238559 0.96233064 0.9623348  0.96234419 0.1394799 ]


In [6]:
x_train,
y_train

array([[0.96238559, 0.96233064, 0.9623348 , 0.96234419, 0.1394799 ],
       [0.96232033, 0.96227354, 0.96230217, 0.9623034 , 0.15839243],
       [0.9622877 , 0.96224907, 0.9622777 , 0.9622463 , 0.08274232],
       ...,
       [0.95714845, 0.95710195, 0.95699951, 0.95709881, 0.20094563],
       [0.95707503, 0.95710195, 0.95708925, 0.95713144, 0.09692671],
       [0.95712398, 0.9570938 , 0.95707293, 0.95710696, 0.17730497]])

In [7]:
from sklearn.model_selection import train_test_split

x_Train, xTest, y_Train, yTest = train_test_split(x_train, y_train, test_size = 0.01, random_state = 0)

# x_Train, xTest

# DESIGN MODEL DNN ARCHITECTURE

In [ ]:
from keras.layers import *

epoch = 1000
batch_size = 96

model = keras.models.Sequential([
    Dense(5, input_dim=5),
    Flatten(),
    Dense(32, activation=tf.nn.leaky_relu),
    Dense(32, activation=tf.nn.leaky_relu),
    Dense(32, activation=tf.nn.leaky_relu),
    Dense(32, activation=tf.nn.leaky_relu),
    Dropout(0.8),
    Dense(batch_size, activation=tf.nn.leaky_relu),
    Dense(batch_size, activation=tf.nn.leaky_relu),
    Dense(batch_size, activation=tf.nn.leaky_relu),
    Dropout(0.8),
    Dense(batch_size, activation=tf.nn.leaky_relu),
    Dense(batch_size, activation=tf.nn.leaky_relu),
    Dense(batch_size, activation=tf.nn.leaky_relu),
    Dropout(0.8),
    Dense(32, activation=tf.nn.leaky_relu),
    Dense(32, activation=tf.nn.leaky_relu),
    Dense(32, activation=tf.nn.leaky_relu),
    Dense(32, activation=tf.nn.leaky_relu),
    Dropout(0.8),
    Dense(5)
])

model.compile(
    optimizer = 'adam', 
    loss = 'mse',
    metrics = [
               'accuracy'
    ]
)

print('Processing...')
t = Timer()
t.start()
train = model.fit(
    x_Train,
    y_Train,
    epochs = epoch,
    verbose = 0,
    # batch_size = batch_size,
    # validation_split = 0.5
)
t.stop()

score = model.evaluate(x_train, y_train)

model.save('/model/')

Processing...


In [ ]:
print("Num of Epoch: ", epoch)
print("AVG Training Accuracy", np.average(train.history['accuracy']))
print("AVG Training Loss", np.average(train.history['loss']))
print("Data Size: ", sys.getsizeof(df))

model.summary()

# PLOTTING LOSS AND ACCURACY

In [ ]:
e = e = [x / epoch for x in range(epoch)] # [*range(0, epochs, 1)]

plt.plot(
  e,
  train.history['accuracy'],
  label='DNN Training Accuracy ({:.4f})'.format(np.average(train.history['accuracy']))
)

plt.plot(
  e,
  train.history['loss'],
  label='DNN Training Loss ({:.4f})'.format(np.average(train.history['loss']))
)
plt.xlabel('Iteration Steps')
plt.ylabel('Metrics Value')
plt.title('DNN Training Accuracy, Loss')
plt.legend(loc=4, prop={'size': 8})

# Predicting

In [ ]:
idx = 20

pred = np.array([
        x_Train[idx]
])

#calculating the accuracy from its result 
abs = np.absolute(
      np.subtract(
          model.predict(pred)[0],
          y_Train[idx]
      )
)
loss = np.divide(abs, y_Train[idx])
acc = np.absolute(
      np.average(1 - loss)
)

print('Predictive: ', model.predict(pred)[0])
print('Target: ', y_Train[idx])

print("AVG Testing Accuracy", acc)

In [ ]:
idx = len(xTest)

plot_target_v1 = np.array([])
plot_predictive_v1 = np.array([])

for i in range(idx):
    test = model.predict(np.array([xTest[i]]))[0]

    plot_target_v1 = np.append(plot_target_v1, y_Train[i][0])
    plot_predictive_v1 = np.append(plot_predictive_v1, test[0])

In [ ]:
e = e = [x / idx for x in range(idx)]

plt.plot(
  e,
  plot_target_v1,
  label='Desired V1'
)

plt.plot(
  e,
  plot_predictive_v1,
  label='Predictive V1'
)
plt.xlabel('Iteration Steps')
plt.ylabel('Metrics Value')
plt.title('DNN Testing')
plt.legend(loc=4, prop={'size': 8})

In [ ]:
#@title Select Date
date_input = '2020-10-13' #@param {type:"date"}

slider_hour = 3 #@param {type:"slider", min:0, max:24, step:1}

slider_minutes = 48 #@param {type:"slider", min:0, max:60, step:1}

print(date_input)
print(slider_hour)
print(slider_minutes)

In [ ]:
#reducing unbalance values

pred = np.array([
        int(date_input.split('-')[0]) / 2021, 
        int(date_input.split('-')[1]) / 12, 
        int(date_input.split('-')[2]) / 31,
        slider_hour / 24,
        slider_minutes / 60
])

max_v1 = 1.22586 #in csv max_v1
max_v2 = 1.22593	
max_v3 = 1.2258
max_v4 = 1.22584	
max_v5 = 423


print("Predictive ", pred)
print("Target ", y_Train[10])

# print("Predictive V1: ", pred[0] * max_v1)
# print("Predictive V2: ", pred[1] * max_v2)
# print("Predictive V3: ", pred[2] * max_v3)
# print("Predictive V4: ", pred[3] * max_v4)
# print("Predictive V5: ", pred[4] * max_v5)

# print('Target V1: ', y_Train[10][0] * max_v1)
# print('Target V2: ', y_Train[10][1] * max_v2)
# print('Target V3: ', y_Train[10][2] * max_v3)
# print('Target V4: ', y_Train[10][3] * max_v4)
# print('Target V5: ', y_Train[10][4] * max_v5)